In [99]:
# Imports and plotting setups
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy

import sys, os
sys.path.insert(0, '../../')
import antenna_toolbox as ant
sys.path.insert(0, '../')

import math_funcs

from cycler import cycler

default_cycler = (cycler(color=['#4477AA', '#EE6677', '#228833', '#CCBB44', '#66CCEE', '#AA3377', '#BBBBBB']) +
                cycler(linestyle=['-', '--', ':', '-.', '-', '--', ':']))

plt.rc('lines', linewidth=1)
plt.rc('axes', prop_cycle=default_cycler)

plt.rcParams["font.family"] = "Times"
plt.rcParams["font.size"] = 8

plt.rcParams['figure.figsize'] = (3.5, 2.5)
plt.rcParams['figure.dpi'] = 600

plt.rcParams['text.usetex'] = True

In [100]:
# Constants
c_0	= 299792458 # m/s
e = 2.718281828
epsilon_0 = 8.854E-12
mu_0 = 1.257E-06
k = 1.38E-23

In [101]:
# Howell 2021 Model parameters
T_surf = 104 # K
T_cond_base = 273.13 # K
brittle_cryosphere_depth = 50 * 1000 # m
surface_background_radiation_temperature = 10**4 # K

In [102]:
# Mission Parameters
max_BER = 10**-5
bit_rate = 1e3 # bps
link_BW = 3.43e3 # Hz

In [103]:
# Engineering/Assumed Parameters
transmitted_power = 1 # W

def antenna_directivity(T):
    directivity_in_273K_ice = 4.46 # dB
    directivity_in_100K_ice = 4.64 # dB
    m, b = math_funcs.linear_fit(
        100, math_funcs.db_2_power(directivity_in_100K_ice), 
        273, math_funcs.db_2_power(directivity_in_273K_ice))
    return math_funcs.power_2_db(m * T + b)

def radiation_efficiency(T):
    rad_eff_in_273K_ice = 0.228 # dB
    rad_eff_in_100K_ice = 0.223 # dB
    m, b = math_funcs.linear_fit(
        100, rad_eff_in_100K_ice, 
        273, rad_eff_in_273K_ice)
    return m * T + b

matching_efficiency = 0.952
carrier_frequency = 413*10**6 # Hz

In [104]:
# Propagation path modeling equations
def temperature_at_depth(depth):
    m, b = math_funcs.linear_fit(
        0, T_surf, 
        brittle_cryosphere_depth, T_cond_base)

    return m * depth + b

def ice_epsilon_relative(T):
    T = T - 273.13 # convert from Kelvin to Celsius
    real = 3.1884 + 0.00091*T
    imag = 10**(-3.0129 + 0.0123*T)
    return real - 1j*imag

def ice_wave_number(T):
    return 2 * np.pi * carrier_frequency * np.sqrt(epsilon_0 * mu_0) * np.sqrt(ice_epsilon_relative(T))

def simple_linear_path_loss(path_length, k_s):
    return np.e**(-2 * path_length * np.imag(k_s))

def wavelength_column_linear_path_loss_calc(top_column, bottom_column):
    approx_wavelength = 0.5
    depths = np.arange(top_column, bottom_column + approx_wavelength, approx_wavelength)
    total_path_loss = 1 # real units of power
    for col_depth in depths:
        total_path_loss *= simple_linear_path_loss(
            approx_wavelength, ice_wave_number(col_depth))
    return total_path_loss


def received_power(T_puck, T_path, path_length):
    k_s = ice_wave_number(T_path)
    #wavelength = np.pi * 2 / np.real(k_s)
    if path_length < 1:
        space_path_loss = 1
    else:
        space_path_loss = (1 / (2 * np.real(k_s) * path_length))**2
    linear_path_loss = simple_linear_path_loss
    
    return transmitted_power * matching_efficiency**2 * radiation_efficiency(T_puck)**2\
        * space_path_loss * linear_path_loss * antenna_directivity(T_puck)**2

def noise_power(T_puck, T_path_to_surface, puck_depth):
    k_s = ice_wave_number(T_path_to_surface)
    linear_path_loss = simple_linear_path_loss(puck_depth, k_s)
    return link_BW * k * temperature_at_depth(puck_depth) \
        + radiation_efficiency(T_puck) * link_BW * k * linear_path_loss * surface_background_radiation_temperature

def CNR_per_bit(received_power, noise_power):
    return (bit_rate/link_BW) * (received_power/noise_power)

def bit_error_rate(CNR_per_bit):
    if CNR_per_bit > 4*np.log(2):
        return np.e**(-1 * 1 * (CNR_per_bit - 2 * np.log(2)))
    else:
        return np.e**(-1 * 2 * ((np.sqrt(CNR_per_bit) - np.sqrt(np.log(2)))**2))

def bit_error_rate_of_next_receiver(last_puck_depth, next_puck_path_length, debug=False):
    puck_depth = last_puck_depth + next_puck_path_length
    T_puck = temperature_at_depth(puck_depth)
    T_path = temperature_at_depth(puck_depth / 2)

    P_r = received_power(T_puck, T_path, next_puck_path_length)
    P_n = noise_power(T_puck, T_path, puck_depth)

    BER = bit_error_rate(CNR_per_bit(P_r, P_n))
    if debug:
        return BER, puck_depth, T_puck, T_path, P_r, P_n
    else:
        return BER

def optimize_puck_depth(last_puck_depth):
    # Goal function
    goal_func = lambda next_puck_path_length: bit_error_rate_of_next_receiver(last_puck_depth, next_puck_path_length) - max_BER

    # Using a binary search optimization (since I know the bit error rate monotonically increases with depth)
    return scipy.optimize.bisect(goal_func, 1, brittle_cryosphere_depth - last_puck_depth)

x = [0, 1, 10, 1000, 8000, 50e3]
ice_wave_number(104), ice_epsilon_relative(104), \
    [10*np.log10(received_power(104, 104, i)) for i in x], \
    [10*np.log10(noise_power(104, 104, i)) for i in x], \
    [10*np.log10(received_power(104, 104, i)/ noise_power(104, 104, i)) for i in x], \
    [10*np.log10(CNR_per_bit(received_power(104, 104, i), noise_power(104, 104, i))) for i in x], \
    [bit_error_rate_of_next_receiver(0, i) for i in x]


((15.080310882041475-2.0049121376258393e-05j),
 (3.0344917000000002-8.06865228237526e-06j),
 [-0.13394092667278729,
  -29.722572588457655,
  -49.721005288357134,
  -89.54860227729893,
  -106.39139082783734,
  -114.9949240389161],
 [-159.565108817594,
  -159.5649361380532,
  -159.56338203091394,
  -159.39252328719277,
  -158.18854660970604,
  -150.9847313323022],
 [159.43116789092122,
  129.84236354959558,
  109.8423767425568,
  69.84392100989385,
  51.79715578186871,
  35.98980729338611],
 [154.0782266904935,
  124.48942234916785,
  104.4894355421291,
  64.49097980946615,
  46.44421458144101,
  30.6368660929584],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

In [105]:
P_a = link_BW * k * temperature_at_depth(1000)
k_s = ice_wave_number(104)
linear_path_loss = np.e**(-2 * 1000 * np.imag(k_s))
P_b = radiation_efficiency(104) * link_BW * k * linear_path_loss * surface_background_radiation_temperature


10*np.log10(P_a) + 30, 10*np.log10(P_b) + 30, 10*np.log10(P_a + P_b)

(-142.93892878451192, -129.5888239774972, -159.39252328719277)

In [106]:
root = optimize_puck_depth(0)
bit_error_rate_of_next_receiver(0, root, True)
# 10 * np.log10(received_power(104, 104, 1e3))
# radiation_efficiency(104)**2

ValueError: f(a) and f(b) must have different signs

In [ ]:
# Loop to calculate the number of pucks and all of their stats
total_depth = 0
inital_guess = 2000
puck_depths = [0]
i = 0

while total_depth < brittle_cryosphere_depth:
    depth_of_next_reciever = puck_depths[i] - 
    T_puck = temperature_at_depth(depth_of_next_reciever)
    T_path_to_surface = 
    max_depth_of_next_reciever = 
    
    total_depth += max_depth_of_next_reciever 

(3.187217-0.0009671659536293614j)